In [1]:
import joblib

import pandas as pd
import numpy as np
import string

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

utility function

In [ ]:
from utils import clean_text

Load data

In [3]:
movie = pd.read_csv('imdb_labelled.txt', delimiter = '\t', names=["reviews", "rating"])
movie.head(4)

,reviews,rating
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0


In [4]:
movie['labels'] = movie["rating"].apply(lambda x: "Positive" if x else "Negative")
movie.head(4) 

,reviews,rating,labels
0,"A very, very, very slow-moving, aimless movie ...",0,Negative
1,Not sure who was more lost - the flat characte...,0,Negative
2,Attempting artiness with black & white and cle...,0,Negative
3,Very little music or anything to speak of.,0,Negative


In [5]:
movie.isna().sum()

reviews    0
rating     0
labels     0
dtype: int64

In [6]:
movie['length'] = movie['reviews'].apply(lambda x: len(str(x).split(' ')))
movie.head()

,reviews,rating,labels,length
0,"A very, very, very slow-moving, aimless movie ...",0,Negative,15
1,Not sure who was more lost - the flat characte...,0,Negative,21
2,Attempting artiness with black & white and cle...,0,Negative,33
3,Very little music or anything to speak of.,0,Negative,10
4,The best scene in the movie was when Gerardo i...,1,Positive,23


In [7]:
movie['length'].unique()

array([  15,   21,   33,   10,   23,   22,    5,   17,   12,    8,   13,
          6,   18,   25,   19,  868,   14,   16,    9,    7,   26,   35,
         11,    4,   27,   28,    3,   20,   37,   24,   36,  201, 1393,
         49,   30,  306,   31,   32,   44,   56,   45,   34,   38,   64,
         58,   40,   47,   29,   73,   41,   39,   55,   53,   42,  800])

In [8]:
unique_lengths = movie['length'].unique()
plt.bar(np.arange(len(unique_lengths)), unique_lengths)

NameError: name 'plt' is not defined

In [11]:
X = movie['reviews']
y = movie['rating']

In [12]:
token, X_test, y_train, y_test = train_test_split(X,y, test_size = .40, random_state = 60)

In [13]:
token.shape, token.to_list()[:4]

((448,),
 ['I am so tired of clichés that is just lazy writing, and here they come in thick and fast.  ',
  'Nothing short of magnificent photography/cinematography in this film.  ',
  'The acting was bad, the dialogs were extremely shallow and insincere.  ',
  "The soundtrack wasn't terrible, either.  "])

training with count vectprizer

In [25]:
cv_NB = CountVectorizer(stop_words = 'english')
vectorizer.fit(token)


X_train_transform = (vectorizer.transform(token).toarray() > 0).astype(int)
X_test_transform = (vectorizer.transform(X_test).toarray() > 0).astype(int)

print("X_test_transform shape = ", X_test_transform.shape)
print("X_train_transform shape = ", X_train_transform.shape)

X_test_transform shape =  (300, 1888)
X_train_transform shape =  (448, 1888)


In [26]:
np.unique(X_test_transform)

array([0, 1])

In [27]:
nb = CategoricalNB()
nb.fit(X_train_transform, y_train)

CategoricalNB()

In [28]:
train_accuracy = nb.score(X_train_transform, y_train)
print(f"Training accuracy is {train_accuracy*100:.2f}%")

Training accuracy is 94.20%


In [29]:
np.unique(X_test_transform)

array([0, 1])

In [31]:
joblib.dump(cv_NB, "models/categorical_naive_bayes_with_count_vectorizer.joblib")

In [32]:
test_accuracy = nb.score(X_test_transform, y_test.values)

print(f"Testing accuracy is {test_accuracy*100:.2f}%")

Testing accuracy is 75.67%


In [33]:
from sklearn.metrics import classification_report, confusion_matrix

y_predicted = nb.predict(X_test_transform)
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.70      0.94      0.80       155
           1       0.89      0.57      0.69       145

    accuracy                           0.76       300
   macro avg       0.79      0.75      0.75       300
weighted avg       0.79      0.76      0.75       300



In [34]:
pd.DataFrame(data=confusion_matrix(y_test, y_predicted, labels=[1, 0]), index=[1, 0], columns=[1, 0])

,1,0
1,82,63
0,10,145


training with tfidf

In [38]:
tfidf = TfidfVectorizer(stop_words = 'english')
vectorizer.fit(token)


X_train_transform = (vectorizer.transform(token).toarray() > 0).astype(int)
X_test_transform = (vectorizer.transform(X_test).toarray() > 0).astype(int)

print("X_test_transform shape = ", X_test_transform.shape)
print("X_train_transform shape = ", X_train_transform.shape)

X_test_transform shape =  (300, 1888)
X_train_transform shape =  (448, 1888)


In [39]:
nb = CategoricalNB()
nb.fit(X_train_transform, y_train)

CategoricalNB()

In [ ]:
joblib.dump(tfidf, "models/categorical_naive_bayes_with tfidf_vectorizer.joblib")

In [40]:
train_accuracy = nb.score(X_train_transform, y_train)
print(f"Training accuracy is {train_accuracy*100:.2f}%")

Training accuracy is 94.20%


In [41]:
from sklearn.metrics import classification_report, confusion_matrix

y_predicted = nb.predict(X_test_transform)
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.70      0.94      0.80       155
           1       0.89      0.57      0.69       145

    accuracy                           0.76       300
   macro avg       0.79      0.75      0.75       300
weighted avg       0.79      0.76      0.75       300

